# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 9 2022 9:51AM,243723,15,30009809,"Alliance Pharma, Inc.",Released
1,Aug 9 2022 9:51AM,243723,15,30009808,"Alliance Pharma, Inc.",Released
2,Aug 9 2022 9:51AM,243723,15,30009810,"Alliance Pharma, Inc.",Released
3,Aug 9 2022 9:51AM,243723,15,30009811,"Alliance Pharma, Inc.",Released
4,Aug 9 2022 9:51AM,243723,15,30009812,"Alliance Pharma, Inc.",Released
5,Aug 9 2022 9:51AM,243723,15,30009814,"Alliance Pharma, Inc.",Released
6,Aug 9 2022 9:51AM,243723,15,30009815,"Alliance Pharma, Inc.",Released
7,Aug 9 2022 9:51AM,243723,15,30009813,"Alliance Pharma, Inc.",Released
8,Aug 9 2022 9:51AM,243723,15,30009816,"Alliance Pharma, Inc.",Released
9,Aug 9 2022 9:51AM,243723,15,30009817,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,243743,Released,11
32,243744,Released,15
33,243746,Released,1
34,243747,Released,1
35,243749,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243743,NaN,NaN,11.0
243744,NaN,NaN,15.0
243746,NaN,NaN,1.0
243747,NaN,NaN,1.0
243749,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243677,0.0,0.0,1.0
243682,0.0,0.0,1.0
243687,13.0,1.0,1.0
243695,0.0,1.0,0.0
243702,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243677,0,0,1
243682,0,0,1
243687,13,1,1
243695,0,1,0
243702,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243677,0,0,1
1,243682,0,0,1
2,243687,13,1,1
3,243695,0,1,0
4,243702,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243677,,,1
1,243682,,,1
2,243687,13,1,1
3,243695,,1,
4,243702,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 9 2022 9:51AM,243723,15,"Alliance Pharma, Inc."
108,Aug 9 2022 9:49AM,243749,19,"GUSA Granules USA, Inc."
109,Aug 9 2022 9:46AM,243747,19,"GUSA Granules USA, Inc."
110,Aug 9 2022 9:46AM,243724,15,"Brookfield Pharmaceuticals, LLC"
149,Aug 9 2022 9:38AM,243739,20,"Exact-Rx, Inc."
156,Aug 9 2022 9:32AM,243744,20,Alumier Labs Inc.
171,Aug 9 2022 9:27AM,243746,10,"Nextsource Biotechnology, LLC"
172,Aug 9 2022 9:15AM,243743,10,Bio-PRF
183,Aug 9 2022 9:06AM,243742,12,Hush Hush
184,Aug 9 2022 9:00AM,243740,19,Else Nutrition


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 9 2022 9:51AM,243723,15,"Alliance Pharma, Inc.",,,108
1,Aug 9 2022 9:49AM,243749,19,"GUSA Granules USA, Inc.",,,1
2,Aug 9 2022 9:46AM,243747,19,"GUSA Granules USA, Inc.",,,1
3,Aug 9 2022 9:46AM,243724,15,"Brookfield Pharmaceuticals, LLC",,,39
4,Aug 9 2022 9:38AM,243739,20,"Exact-Rx, Inc.",,,7
5,Aug 9 2022 9:32AM,243744,20,Alumier Labs Inc.,,,15
6,Aug 9 2022 9:27AM,243746,10,"Nextsource Biotechnology, LLC",,,1
7,Aug 9 2022 9:15AM,243743,10,Bio-PRF,,,11
8,Aug 9 2022 9:06AM,243742,12,Hush Hush,,,1
9,Aug 9 2022 9:00AM,243740,19,Else Nutrition,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 9:51AM,243723,15,"Alliance Pharma, Inc.",108,,
1,Aug 9 2022 9:49AM,243749,19,"GUSA Granules USA, Inc.",1,,
2,Aug 9 2022 9:46AM,243747,19,"GUSA Granules USA, Inc.",1,,
3,Aug 9 2022 9:46AM,243724,15,"Brookfield Pharmaceuticals, LLC",39,,
4,Aug 9 2022 9:38AM,243739,20,"Exact-Rx, Inc.",7,,
5,Aug 9 2022 9:32AM,243744,20,Alumier Labs Inc.,15,,
6,Aug 9 2022 9:27AM,243746,10,"Nextsource Biotechnology, LLC",1,,
7,Aug 9 2022 9:15AM,243743,10,Bio-PRF,11,,
8,Aug 9 2022 9:06AM,243742,12,Hush Hush,1,,
9,Aug 9 2022 9:00AM,243740,19,Else Nutrition,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 9:51AM,243723,15,"Alliance Pharma, Inc.",108,,
1,Aug 9 2022 9:49AM,243749,19,"GUSA Granules USA, Inc.",1,,
2,Aug 9 2022 9:46AM,243747,19,"GUSA Granules USA, Inc.",1,,
3,Aug 9 2022 9:46AM,243724,15,"Brookfield Pharmaceuticals, LLC",39,,
4,Aug 9 2022 9:38AM,243739,20,"Exact-Rx, Inc.",7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 9:51AM,243723,15,"Alliance Pharma, Inc.",108.0,NaN,NaN
1,Aug 9 2022 9:49AM,243749,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
2,Aug 9 2022 9:46AM,243747,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Aug 9 2022 9:46AM,243724,15,"Brookfield Pharmaceuticals, LLC",39.0,NaN,NaN
4,Aug 9 2022 9:38AM,243739,20,"Exact-Rx, Inc.",7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 9:51AM,243723,15,"Alliance Pharma, Inc.",108.0,0.0,0.0
1,Aug 9 2022 9:49AM,243749,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
2,Aug 9 2022 9:46AM,243747,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Aug 9 2022 9:46AM,243724,15,"Brookfield Pharmaceuticals, LLC",39.0,0.0,0.0
4,Aug 9 2022 9:38AM,243739,20,"Exact-Rx, Inc.",7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3168390,118.0,1.0,0.0
102,974861,4.0,0.0,0.0
12,243742,1.0,0.0,0.0
15,487447,147.0,0.0,0.0
16,243703,0.0,1.0,0.0
19,974931,2.0,2.0,0.0
20,974878,35.0,4.0,13.0
21,731163,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3168390,118.0,1.0,0.0
1,102,974861,4.0,0.0,0.0
2,12,243742,1.0,0.0,0.0
3,15,487447,147.0,0.0,0.0
4,16,243703,0.0,1.0,0.0
5,19,974931,2.0,2.0,0.0
6,20,974878,35.0,4.0,13.0
7,21,731163,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,118.0,1.0,0.0
1,102,4.0,0.0,0.0
2,12,1.0,0.0,0.0
3,15,147.0,0.0,0.0
4,16,0.0,1.0,0.0
5,19,2.0,2.0,0.0
6,20,35.0,4.0,13.0
7,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,118.0
1,102,Released,4.0
2,12,Released,1.0
3,15,Released,147.0
4,16,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,19,20,21
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0
Executing,1.0,0.0,0.0,0.0,1.0,2.0,4.0,0.0
Released,118.0,4.0,1.0,147.0,0.0,2.0,35.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0
1,Executing,1.0,0.0,0.0,0.0,1.0,2.0,4.0,0.0
2,Released,118.0,4.0,1.0,147.0,0.0,2.0,35.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0
1,Executing,1.0,0.0,0.0,0.0,1.0,2.0,4.0,0.0
2,Released,118.0,4.0,1.0,147.0,0.0,2.0,35.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()